### Hive assignment. Task1

The purpose of this task is to create an external table on the posts data of the stackoverflow.com website.

Create your own database and `'use'` it. Create external table `'posts_sample_external'` over the sample dataset with posts in `'/data/stackoverflow_1000'` directory. Create managed table 'posts_sample' and populate with the data from the external table. `'Posts_sample'` table should be partitioned by year and by month of post creation. Provide output of query which selects lines number per each partition in the format:

`year <tab> month <table> lines count`

where year in `YYYY` format and month in `YYYY-MM` format. The result is the 3th line of the last query output.

Example:

`2008 2008-07 123`

In [63]:
%%writefile query.hql

-- CREATE DATABASE IF NOT EXISTS jovyan;
-- USE jovyan;


Overwriting query.hql


In [64]:
%%writefile -a query.hql

-- Create posts_sample_external table

DROP TABLE IF EXISTS `posts_sample_external`;
CREATE EXTERNAL TABLE IF NOT EXISTS `posts_sample_external`(
  `id` int,
  `post_type_id` tinyint,
  `date` string,
  `owner_user_id` int,
  `parent_id` int,
  `score` int,
  `favorite_count` int,
  `tags` string
)
PARTITIONED BY ( 
  `year` string, 
  `month` string
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.RegexSerDe'
WITH SERDEPROPERTIES (
  "input.regex" = '^<row.*?(?=.*\\bId=\"(\\d+)\")(?=.*\\bPostTypeId=\"(\\d+)\")(?=.*\\bCreationDate=\"([^"]*)\")(?=.*\\bOwnerUserId=\"(\\d+)\")?(?=.*\\bParentId=\"(\\d+)\")?(?=.*\\bScore=\"(-?\\d+)\")(?=.*\\bFavoriteCount=\"(\\d+)\")?(?=.*\\bTags=\"([^"]*)\")?.*',
  "input.regex.case.insensitive" = 'true'
)
STORED AS TEXTFILE
LOCATION
  '/data/stackexchange100/posts'
;


Appending to query.hql


In [65]:
%%writefile -a query.hql

-- Create managed table and fill data

DROP TABLE IF EXISTS `posts_sample`;
CREATE TABLE IF NOT EXISTS `posts_sample`(
  `id` int,
  `post_type_id` tinyint,
  `date` string,
  `owner_user_id` int,
  `parent_id` int,
  `score` int,
  `favorite_count` int,
  `tags` array <string>
)
PARTITIONED BY ( 
  `year` string, 
  `month` string
)
CLUSTERED BY ( 
  `date`
) 
SORTED BY ( 
  id ASC
) 
INTO 8 BUCKETS
STORED AS ORC
;

SET hive.exec.dynamic.partition=true;  
SET hive.exec.dynamic.partition.mode=nonstrict;

INSERT OVERWRITE TABLE `posts_sample`
PARTITION (`year`, `month`)
SELECT
  `id`,
  `post_type_id`,
  `date`,
  `owner_user_id`,
  `parent_id`,
  `score`,
  `favorite_count`,
  split(regexp_replace(`tags`, '(&lt\;|&gt\;$)', ''), '&gt\;') AS `tags`,
  regexp_extract(`date`, '^(\\d{4})', 1) AS `year`,
  regexp_extract(`date`, '^\\d{4}-(\\d{2})', 1) AS `month`
FROM `posts_sample_external`
;


Appending to query.hql


In [66]:
! hive -f query.hql 1>2


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
OK
Time taken: 1.284 seconds
OK
Time taken: 0.304 seconds
OK
Time taken: 0.14 seconds
OK
Time taken: 0.138 seconds
Query ID = jovyan_20171101124040_b6e194d1-8a84-4596-ba5d-580555dc3875
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks is set to 0 since there's no reduce operator
Starting Job = job_1509524427668_0010, Tracking URL = http://1abfcb4c4022:8088/proxy/application_1509524427668_0010/
Kill Command = /opt/hadoop/bin/hadoop job  -kill job_1509524427668_0010
Hadoop job information for Stage-1: number of mappers: 2; number of reducers: 0
2017-11-01 12:40:58,113 Stage-1 map = 0%,  reduce = 0%
2017-11-01 12:41:15,175 Stage-1 map = 5%,  reduce = 0%, Cumulative CPU 18.46 sec
2017-11-01 12:41:16,214 Stage-1 map = 12%,  reduce = 0%, Cumulative CPU 34.61 sec
2017-11-01 12:41:21,413 Stage-1 map = 21%,  reduce = 0%, Cumulative CPU 47.27 sec
201

In [1]:
! hive --database mskorokhod -e 'show tables'


Logging initialized using configuration in jar:file:/opt/cloudera/parcels/CDH-5.11.0-1.cdh5.11.0.p0.34/jars/hive-common-1.1.0-cdh5.11.0.jar!/hive-log4j.properties
OK
Time taken: 1.139 seconds
OK
posts
users
Time taken: 0.184 seconds, Fetched: 2 row(s)
